In [1]:
import os
from dotenv import load_dotenv, find_dotenv

In [2]:
import nest_asyncio
nest_asyncio.apply()

import warnings
warnings.filterwarnings("ignore")

from llama_index.core import (
    PropertyGraphIndex,
    SimpleDirectoryReader,
    Settings,
    StorageContext,
    load_index_from_storage
)

from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core.indices.property_graph import (
    ImplicitPathExtractor,
    SimpleLLMPathExtractor
)
from llama_index.llms.groq import Groq
from llama_index.embeddings.ollama import OllamaEmbedding

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [3]:
Settings.llm = Groq("llama-3.3-70b-versatile")
Settings.embed_model=OllamaEmbedding(model_name="nomic-embed-text")
documents = SimpleDirectoryReader(input_files=["../docs/MBA_Worth_It.pdf"]).load_data(show_progress=True)

Loading files: 100%|██████████| 1/1 [00:00<00:00,  4.10file/s]


In [4]:
graph_store = Neo4jPropertyGraphStore(
    username="neo4j",
    password="llamaindex",
    url="bolt://localhost:7687",
)

In [5]:
from qdrant_client import QdrantClient, AsyncQdrantClient

client = QdrantClient(
    host="localhost",
    port=6333
)
aclient = AsyncQdrantClient(
    host="localhost",
    port=6333
)

vector_store = QdrantVectorStore(
    collection_name="mba",
    client=client,
    aclient=aclient,
    prefer_grpc=True,
)

In [6]:
index = PropertyGraphIndex.from_documents(
    documents,
    kg_extractors = [
        ImplicitPathExtractor(),
        SimpleLLMPathExtractor(
            num_workers = 4,
            max_paths_per_chunk = 10,
        )
    ],
    show_progress=True,
    property_graph_store=graph_store,
    vector_store=vector_store,
)

Parsing nodes:   0%|          | 0/7 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████| 35/35 [00:03<00:00,  9.90it/s]


In [22]:
query_engine = index.as_query_engine(
    include_text=True,
    similarity_top_k=4
)

In [23]:
retriever = index.as_retriever(include_text=False)
nodes = retriever.retrieve("What are some considerations when deciding whether or not to embark on an MBA?")
for node in nodes:
    print(node.text)

Mba -> Is -> Graduate degree
Mba degree -> Offers -> Professional development
Mba -> Accelerates -> Professional development
Business school -> Offer -> Great time to gain exposure
Business school -> Provide -> Exposure to faculty
Business school -> Provide -> Exposure to peers
Some students -> Come to -> Business school
Business school -> Provides -> Academic benefits
Business school -> Provide -> Exposure to alumni network
Business school -> Requires -> Academic effort
Business school -> May not be -> Right step
Business school -> Offers -> Financial support
Mba -> Is planned for -> Career advancement


In [24]:
query_engine = index.as_query_engine(include_text=True)

response = query_engine.query("What are some reasons to take an MBA?")

In [25]:
from IPython.display import display, Markdown

display(Markdown(str(response)))

One reason to take an MBA is for career advancement. It is also implied that having an MBA can help individuals get to where they want to go in their careers, as it is a part of planning for career goals.

In [26]:
response = query_engine.query("What are some considerations when deciding whether or not to embark on an MBA?")

display(Markdown(str(response)))

Some considerations when deciding whether or not to embark on an MBA include reflecting on your current situation, identifying areas for improvement, and determining what skills you need to acquire. It's essential to think about where you are, where you want to go, and what it will take to get you there. You should also consider whether an MBA will help you achieve your career ambitions and if it's the right step for you. Additionally, you may want to explore different specializations and concentrations within MBA programs to find the best fit for your goals. Ultimately, making the right decision requires careful thought and consideration of your motivations, goals, and circumstances.

In [27]:
response = query_engine.query("What are the right motivations for an MBA?")

display(Markdown(str(response)))

The right motivations for an MBA include a desire for professional development, a wish to attain leadership skills, and a plan for career advancement. It is also important to have a clear understanding of where you are, where you want to go, and what it will take to get there. Additionally, having a couple of years of hands-on work experience before pursuing an MBA can help you figure out what areas you want to focus on and make the most out of your time.

In [33]:
response = query_engine.query("What are the wrong motivations to take an MBA?")

display(Markdown(str(response)))

Empty Response

Why empty?!

In [37]:
response = query_engine.query("Why should anyone do an MBA?")

display(Markdown(str(response)))

An MBA offers several benefits, including a bigger paycheck, access to job opportunities, and a stronger professional network. It also helps individuals hone their skills, diversify their professional network, and make a career pivot. Additionally, an MBA provides access to connections and accelerates professional development. Overall, it can be a valuable investment for career advancement.

In [38]:
response = query_engine.query("Why should anyone not do an MBA?")

display(Markdown(str(response)))

It requires a significant amount of time, money, effort, and hard work. Additionally, around 51% of MBA students end up taking out student loans to finance their education, which can be a significant financial burden. The cost alone is a considerable factor to consider, and it may not be the right decision for everyone.

In [46]:
response = query_engine.query("So how should anyone decide on whether to do an MBA?")

display(Markdown(str(response)))

To decide on whether to do an MBA, one should consider where they are, where they want to go, and what it will take to get them there. This involves thinking about their current situation, their career goals, and the potential benefits and costs of pursuing an MBA. It's also important to look beyond just the cost and consider other factors, such as the potential for career advancement, networking opportunities, and personal development. By weighing these factors and considering their individual circumstances, individuals can make an informed decision about whether an MBA is right for them.

## Vector Store Index

In [43]:
from llama_index.core import VectorStoreIndex, StorageContext

vector_store2 = QdrantVectorStore(
    collection_name="mba_vectors_only",
    client=client,
    aclient=aclient,
    prefer_grpc=True,
)
storage_context = StorageContext.from_defaults(vector_store=vector_store2)
index2 = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context
)

In [44]:
query_engine2 = index2.as_query_engine(similarity_top_k=4)
response = query_engine2.query("What are the wrong motivations to take an MBA?")

display(Markdown(str(response)))

The text does not explicitly state the wrong motivations to take an MBA, but it implies that cost alone should not be the deciding factor. It suggests that considering only the financial aspect may not be a sufficient reason to pursue an MBA. However, it does not provide a clear list of wrong motivations.

In [45]:
response = query_engine2.query("What are some considerations when deciding whether or not to embark on an MBA?")

display(Markdown(str(response)))

Some considerations when deciding whether or not to embark on an MBA include the time, money, and effort required, as well as the potential benefits such as honing skills like influence, empathy, and building business strategies, making a career pivot, accelerating professional development, and expanding and diversifying one's professional network. Additionally, the cost of an MBA should not be the sole deciding factor, as there are alternative ways to finance it, such as fellowships and employer sponsorships.